# [Jupyter]NeuralNetwork.ipynb
Description   : Neural network

신경망을 직접 구현하여 회귀를 활용해보기로 한다.<br>
많은 예시로 사용되고 있는 전복의 나이 예측 단층 퍼셉트론 신경망을 구현해보자.

In [8]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import os

np.random.seed(1500)
# 경로 초기화
os.chdir(r"C:\Users\TitusChoi\Desktop\Library\CodeLion\AI")

In [9]:
# Hyperparameter
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

In [10]:
# Main function
def main_exec(epochs = 10, mb_size = 10, report = 1, tr = 0.7): # 학습 횟수, 미니 배치 사이즈, 학습 리포트, 훈련 비율(데이터 전부 훈련시키지 않고 그 중 훈련 비율만큼 훈련, 나머지는 시험 비율이 된다.)
    load_dataset() # 데이터 셋 여는 함수
    init_model() # 가중치와 편향 초기화 함수
    train_and_test(epochs, mb_size, report, lr) # 학습 및 신경망 성능 테스트 함수

In [13]:
# Data load, one-hot encoding
def load_dataset():
    # Loading datasets
    with open('./datasets/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 첫 번째 index 건너뛰고 none으로 반환
        rows = []
        for row in csvreader:
            rows.append(row)

    # Global Variable
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1 # 독립변수의 크기와 종속변수의 크기
    
    # Buffer
    data = np.zeros([len(rows), input_cnt + output_cnt])

    # One-hot encoding
    for index, row in enumerate(rows):
        if row[0] == 'I' : data[index, 0] = 1
        elif row[0] == 'M' : data[index, 1] = 1
        elif row[0] == 'F' : data[index, 2] = 1
        data[index, 3:] = row[1:]

[[ 0.      1.      0.     ...  0.101   0.15   15.    ]
 [ 0.      1.      0.     ...  0.0485  0.07    7.    ]
 [ 0.      0.      1.     ...  0.1415  0.21    9.    ]
 ...
 [ 0.      1.      0.     ...  0.2875  0.308   9.    ]
 [ 0.      0.      1.     ...  0.261   0.296  10.    ]
 [ 0.      1.      0.     ...  0.3765  0.495  12.    ]]


In [14]:
# Parameter Initialization
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [18]:
# Train and Test / Result check
def train_and_test(epochs, mb_size, report, tr):
    steps = arrange_data(mb_size, tr)           # 반환하는 값은 미니배치가 몇 덩어리(스텝)으로 쪼개지는지?
    test_x, test_y = get_test_data()               # 테스트 데이터에 대한 독립변수와 종속변수를 얻어내는 함수

    # epochs는 외부 for문으로 돌림
    # 미니배치는 내부 for문으로 돌리기 때문에 시간복잡도 증가
    for epoch in range(epochs):
        losses, accuracies = [], [] # epochs당 손실함수와 정확도 for 평균
        for n in range(steps):
            train_x, train_y = get_train_data(mb_size, n) # 미니배치 사이즈와 스텝의 수만큼 개별값을 받아 학습데이터의 독립, 종속변수 반환
            loss, accuracy = run_train(train_x, train_y)
            losses.append(loss)
            accuracies.append(accuracy)
        
        if report > 0 and (epoch + 1) % report == 0:
            accuracy = run_test(test_x, test_y)
            print('Epoch {} : train - loss = {:5.3f}, accuracy = {:5.3f}, Test={:5.3f}'\
                .format(epoch + 1, np.mean(losses), np.mean(accuracies), accuracy))
    
    final_accuracy = run_test(test_x, test_y)
    print("\n 최종 테스트 결과 : final accuracy = {:5.3f}".format(final_accuracy))